# Model Evaluation

In [441]:
# Import required libraries
import numpy as np
import pandas as pd
import os
from gensim.models import FastText

import mlflow

import json
import requests
import boto3

import streamlit as st

import warnings
warnings.filterwarnings("ignore")

## Prep

In [389]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [390]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136 
user_item = table.get_item(Key={"user_id": int(user_id)})["Item"]

liked_idx = list(map(int, user_item["liked_idx"].keys()))
disliked_idx = list(map(int, user_item["disliked_idx"].keys()))

data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

In [448]:
import random
random.seed(42)

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("FastText-Model")

# Load params
params = load_params()

# Load metadata
with open("../experiment_metadata.json", "r") as f:
    metadata = json.load(f)

with mlflow.start_run(run_id=metadata["run_id"]):
    # Load raw and mlflow model
    model = mlflow.pyfunc.load_model(metadata["local_uri"])
    fasttext_model = FastText.load(metadata["model_params"]["model_path"])

    # Model evaluation
    k = 10
    n_ing = 3

    if k > params["model_service"]["n_recs"]:
        k = params["model_service"]["n_recs"]
        print(f"Adjusted k to {k}")

    precisions = []
    recalls = []
    ndcg_scores = []

    unique_ingredients = data.ingredients.explode().drop_duplicates().tolist()
    for i in range(10):
        # Get random ingredients as query
        q_sample = random.sample(unique_ingredients, n_ing)

        # Grab recommendations from the query
        recs = data.iloc[model.predict(q_sample)][:params["model_service"]["n_recs"]]

        # Compute similarities between query and recommendations ingredients
        recs_sims = []
        for recipe_ings in recs.ingredients:
            recipe_sims = []
            for ing in recipe_ings:
                ing_sims = np.array([fasttext_model.wv.similarity(ing, q_ing) for q_ing in q_sample]) # Similarity values between a recipe ingredient and query ingredients
                max_sim = ing_sims.max()
                recipe_sims.append(max_sim)
            recipe_sims = np.array(recipe_sims) # Best similarity values between recipe ingredients and query ingredients

            # print(f"Ingredients: {ingredients}")
            # print(f"Similarities: {np.digitize(sims.mean(), bins=[0, 0.25, 0.5, 0.75, 1], right=True)}")
            # recs_sims.append(1 if sims.mean() > 0.5 else 0)

            recs_sims.append(recipe_sims.mean()) # Append overall similarity between recipe ingredients and query ingredients

            # recs_sims.append(np.digitize(sims.mean(), bins=[0, 0.25, 0.5, 0.75, 1], right=True)/5)

        recs_sims = np.array(recs_sims)
        
        # Calculate metrics
        # Precision@K
        precisions.append(recs_sims[:k].mean()) # sum@k/k = mean
        # print("Precision@K:", recs_sims[:k].mean()) # Precision@K

        # Recall@K
        recalls.append(recs_sims[:k].sum()/recs_sims.sum()) # sum@k / all relevant (unconstrained by k)
        # print("Recall@K:", recs_sims[:k].sum()/recs_sims.sum()) # Hit@K or Recall@K

        # NDCG@K
        dcg = []
        for i in range(len(recs_sims)):
            dcg.append(recs_sims[i]/np.log2((i+1)+1))

        dcg = np.array(dcg).sum()
        
        idcg = []
        sorted_recs = sorted(recs_sims, reverse=True)
        for i in range(len(sorted_recs)):
            idcg.append(sorted_recs[i]/np.log2((i+1)+1))

        idcg = np.array(idcg).sum() + 1e-10  # Avoid dividing by 0

        ndcg = dcg/idcg
        ndcg_scores.append(ndcg)

    mlflow.log_metric(f"Precision-{k}", np.array(precisions).mean())
    mlflow.log_metric(f"Recall-{k}", np.array(recalls).mean())
    mlflow.log_metric("NDCG", np.array(ndcg_scores).mean())

    print(f"Precision@{k}:", np.array(precisions).mean())
    print(f"Recall@{k}:", np.array(recalls).mean())
    print("NDCG:", np.array(ndcg_scores).mean())

Precision@10: 0.45052662
Recall@10: 0.35978454
NDCG: 0.9650799065472644
🏃 View run spiffy-lamb-327 at: http://127.0.0.1:8080/#/experiments/198067954999234058/runs/d60a2109d99e4d3cab79ffbb21a66c1a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/198067954999234058


In [435]:
text = "honey, salt, pepper, chicken, beef, fish"
query = [ingredient.strip() for ingredient in text.split(",")]
data.iloc[model.predict(query)]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
380,380,Honey Soy Chicken,https://www.justonecookbook.com/honey-soy-sauc...,"[chicken_drumettes, salt, black_pepper, honey,...",7.0,80.0,10.0,30.0,30.0,302.0,...,410.0,195.0,1.0,6.0,180.0,1.0,15.0,1.0,0.0,https://www.justonecookbook.com/wp-content/upl...
6135,6135,Three-Ingredient Grilled Chicken Wings,https://thewoksoflife.com/three-ingredient-gri...,"[chicken_wing, salt, black_pepper]",4.0,320.0,300.0,20.0,0.0,319.0,...,880.0,232.0,1.0,1.0,210.0,1.0,20.0,1.4,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5484,5484,Down A Chicken,https://thewoksoflife.com/how-to-break-down-a-...,[chicken],9.0,10.0,10.0,0.0,0.0,409.0,...,133.0,360.0,0.0,0.0,267.0,3.0,21.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5558,5558,Chicken Thighs,https://thewoksoflife.com/how-to-debone-chicke...,[chicken_thigh],4.0,10.0,10.0,0.0,0.0,211.0,...,74.0,198.0,0.0,0.0,75.0,0.0,8.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
4576,4576,Honey Garlic Chicken Breast,https://www.recipetineats.com/honey-garlic-chi...,"[chicken_breast, salt, pepper, flour, unsalted...",9.0,12.0,4.0,8.0,0.0,323.0,...,400.0,486.0,0.0,21.0,350.0,2.0,12.0,1.0,166.0,https://www.recipetineats.com/tachyon/2018/09/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,3025,Chocolate Hazelnut No-Bake Cookies,https://minimalistbaker.com/chocolate-hazelnut...,"[hazelnut_meal, medjool_nour_date, cocoa, dair...",4.0,5.0,5.0,0.0,0.0,105.0,...,0.0,0.0,2.0,12.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2895,2895,Chocolate Cherry Almond Milk,https://minimalistbaker.com/chocolate-cherry-a...,"[almond_breeze_original_unsweetened, medjool_d...",3.0,15.0,15.0,0.0,0.0,162.0,...,282.0,0.0,4.6,9.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
1418,1418,Wheat Berries,https://www.loveandlemons.com/how-to-cook-whea...,[dry_wheat_berry],2.0,50.0,10.0,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...
1492,1492,Oat Flour,https://www.loveandlemons.com/oat-flour/,[rolled_oat],2.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...


In [ ]:
metadata["model_params"]

{'server_uri': 'models:/m-1de6ed1b1a484ef49d454f5e9cfc937e',
 'local_uri': '../mlflow/mlartifacts/620532297088492268/models/m-1de6ed1b1a484ef49d454f5e9cfc937e/artifacts',
 'run_id': 'c2ca875abd724656a5cd321d60496ec1',
 'time_logged': '2025-08-25 03:36:49.719654',
 'model_params': {'model_path': '../models/fasttext.model',
  'model_ngram_path': '../models/fasttext.model.wv.vectors_ngrams.npy',
  'recipe_path': '../data/processed_cookbook.pkl',
  'fast_text': {'vector_size': 100, 'window': 5, 'sg': 0, 'epochs': 100},
  'model_scoring': {'w_title': 0.1, 'w_cosine': 0.2, 'w_maxsim': 0.7}}}

In [438]:
import random
random.seed(42)

precisions = []
recalls = []
ndcg_scores = []
k=10

for i in range(10):
    test = random.sample(data.ingredients.explode().drop_duplicates().tolist(), 3)
    
    recs = data.iloc[model.predict(test)][:30] # Give 30 Recs
    recs_sims = []
    for ingredients in recs.ingredients:
        sims = []
        for ing in ingredients:
            sim = np.array([fasttext_model.wv.similarity(ing, i) for i in test]).max()
            sims.append(sim)
        sims = np.array(sims)
        # print(f"Ingredients: {ingredients}")
        # print(f"Similarities: {np.digitize(sims.mean(), bins=[0, 0.25, 0.5, 0.75, 1], right=True)}")
        # recs_sims.append(1 if sims.mean() > 0.5 else 0)
        recs_sims.append(sims.mean())
        # recs_sims.append(np.digitize(sims.mean(), bins=[0, 0.25, 0.5, 0.75, 1], right=True)/5)

    recs_sims = np.array(recs_sims)
    
    precisions.append(recs_sims[:k].mean()) # sum@k/k = mean
    # print("Precision@K:", recs_sims[:k].mean()) # Precision@K

    
    recalls.append(recs_sims[:k].sum()/recs_sims.sum()) # sum@k / all relevant (unconstrained by k)
    # print("Recall@K:", recs_sims[:k].sum()/recs_sims.sum()) # Hit@K or Recall@K

    dcg = []
    for i in range(len(recs_sims)):
        dcg.append(recs_sims[i]/np.log2((i+1)+1))

    dcg = np.array(dcg).sum()
    
    idcg = []
    sorted_recs = sorted(recs_sims, reverse=True)
    for i in range(len(sorted_recs)):
        idcg.append(sorted_recs[i]/np.log2((i+1)+1))

    idcg = np.array(idcg).sum() + 1e-10  # Avoid dividing by 0

    ndcg = dcg/idcg
    ndcg_scores.append(ndcg)

print(f"Precision@{k}:", np.array(precisions).mean())
print(f"Recall@{k}:", np.array(recalls).mean())
print("NDCG:", np.array(ndcg_scores).mean())

Precision@10: 0.45052662
Recall@10: 0.35978454
NDCG: 0.9650799065472644


In [ ]:
Precision@10: 0.43807453
Recall@10: 0.36145052
NDCG: 0.9595688678412845

In [285]:
results

[2.7, 2.6, 2.2, 2.3, 2.6, 2.5, 2.9, 2.2, 2.0, 2.3]

In [283]:
recs_sims

array([2, 2, 3, 2, 2, 2, 2, 4, 2, 2])

In [210]:
ndcg_scores

[0.0,
 0.9999999999747057,
 0.44130740932958,
 0.9999999999747057,
 0.871078543959459,
 0.6309297535083646,
 0.9199753853524705,
 0.9783978399218972,
 0.9848330959282815,
 0.9619213015202485]

In [377]:
random.seed(42)
test = random.sample(data.ingredients.explode().drop_duplicates().tolist(), 5)
# print(test)

k = 10
recs = data.iloc[model.predict(test)][:k]
recs_sims = []
for ingredients in recs.ingredients:
    sims = []
    for ing in ingredients:
        sim = np.array([fasttext_model.wv.similarity(ing, i) for i in test]).max()
        sims.append(sim)
    sims = np.array(sims)
    # print(f"Ingredients: {ingredients}")
    # print(f"Similarities: {sims.mean()}")
    recs_sims.append(sims.mean())
    # recs_sims.append(np.digitize(sims.mean(), bins=np.linspace(0, 1, 10), right=True))
    

dcg = []
for i in range(len(recs_sims)):
    dcg.append(recs_sims[i]/np.log2((i+1)+1))

dcg = np.array(dcg).sum()

idcg = []
sorted_recs = sorted(recs_sims, reverse=True)
for i in range(len(sorted_recs)):
    idcg.append(sorted_recs[i]/np.log2((i+1)+1))

idcg = np.array(idcg).sum() + 1e-10  # Avoid dividing by 0

ndcg = dcg/idcg
ndcg
# recs_sims = np.array(recs_sims)
# print("Hit@K:", recs_sims.mean()) # Hit@K or Recall@K


0.966495008181378

In [372]:
dcg/idcg

0.966495008181378

In [333]:
sims.mean()

0.40144295

In [319]:
dcg = []
for i in range(len(recs_sims)):
    dcg.append(recs_sims[i]/np.log2((i+1)+1))

In [321]:
sims.mean()

0.40144295

In [305]:
data.iloc[model.predict(test)][:k]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
1710,1710,The Gluten-Free Chocolate Cake,https://minimalistbaker.com/the-best-gluten-fr...,"[almond_flour, coconut_sugar_sifted_clumpy, po...",7.0,50.0,20.0,30.0,0.0,293.0,...,356.0,321.0,3.5,17.2,0.0,0.0,79.0,1.2,1.0,https://minimalistbaker.com/wp-content/uploads...
5366,5366,Gluten-Free Dumplings,https://thewoksoflife.com/gluten-free-dumplings/,"[glutenfreeflour, tapioca_starch, glutinous_ri...",9.0,45.0,35.0,10.0,0.0,202.0,...,34.0,22.0,4.0,1.0,0.0,0.0,18.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
2829,2829,3-Ingredient Dark Chocolate Peppermint Mousse,https://minimalistbaker.com/3-ingredient-dark-...,"[coconut_creamfat_milk, dairyfree_dark_chocola...",9.0,257.0,255.0,2.0,0.0,544.0,...,63.0,0.0,4.0,35.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2490,2490,2-Ingredient Coconut Yogurt,https://minimalistbaker.com/easy-2-ingredient-...,"[full_fat_coconut_milk, veganfriendly_probioti...",8.0,360.0,360.0,0.0,0.0,120.0,...,25.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2928,2928,Vegan Mint Chocolate Ice Cream Cake,https://minimalistbaker.com/vegan-mint-chocola...,"[bowl_vegan_chocolate_cake_recipe, vegan_mint_...",6.0,30.0,30.0,0.0,0.0,466.0,...,274.0,0.0,7.0,12.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
3072,3072,Vegan Brownie Chocolate Ice Cream,https://minimalistbaker.com/vegan-brownie-choc...,"[fullfat_coconut_milk, unsweetened_vanilla_alm...",3.0,150.0,150.0,0.0,0.0,208.0,...,0.0,0.0,9.0,15.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2590,2590,3-Ingredient Vegan Chocolate Malt,https://minimalistbaker.com/3-ingredient-vegan...,"[vanilla_bean_coconut_ice_cream, vegan_chocola...",3.0,5.0,5.0,0.0,0.0,303.0,...,115.0,0.0,1.3,35.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2250,2250,Seed Cycling Mixes,https://minimalistbaker.com/seed-cycling-mixes...,"[flax_seed, pumpkin_seed, chia_seed, desiccate...",4.0,10.0,10.0,0.0,0.0,98.0,...,4.0,99.0,2.3,0.4,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
1754,1754,Creamy Raspberry Chip Smoothie,https://minimalistbaker.com/creamy-raspberry-c...,"[dairyfree_milk, raspberry, banana, dairyfree_...",3.0,5.0,5.0,0.0,0.0,257.0,...,93.0,563.0,7.1,15.0,16.0,18.2,258.0,2.2,1.0,https://minimalistbaker.com/wp-content/uploads...
2904,2904,Vegan Roasted Red Pepper Pasta,https://minimalistbaker.com/vegan-roasted-red-...,"[bell_pepper, olive_oil, shallot, garlic, salt...",8.0,60.0,15.0,45.0,0.0,487.0,...,365.0,0.0,10.0,4.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...


array([4, 1, 1, 2, 1, 3, 2, 2, 1, 3, 2, 3, 2])

In [218]:
dcg = []
for i in range(len(recs_sims)):
    dcg.append(recs_sims[i]/np.log2((i+1)+1))

dcg = np.array(dcg).sum()

idcg = []
sorted_recs = sorted(recs_sims, reverse=True)
for i in range(len(sorted_recs)):
    idcg.append(sorted_recs[i]/np.log2((i+1)+1))

idcg = np.array(idcg).sum() + 1e-10  # Avoid dividing by 0

ndcg = dcg/idcg
ndcg

0.8653082041974357

In [110]:
recs

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
1710,1710,The Gluten-Free Chocolate Cake,https://minimalistbaker.com/the-best-gluten-fr...,"[almond_flour, coconut_sugar_sifted_clumpy, po...",7.0,50.0,20.0,30.0,0.0,293.0,...,356.0,321.0,3.5,17.2,0.0,0.0,79.0,1.2,1.0,https://minimalistbaker.com/wp-content/uploads...
5366,5366,Gluten-Free Dumplings,https://thewoksoflife.com/gluten-free-dumplings/,"[glutenfreeflour, tapioca_starch, glutinous_ri...",9.0,45.0,35.0,10.0,0.0,202.0,...,34.0,22.0,4.0,1.0,0.0,0.0,18.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
2829,2829,3-Ingredient Dark Chocolate Peppermint Mousse,https://minimalistbaker.com/3-ingredient-dark-...,"[coconut_creamfat_milk, dairyfree_dark_chocola...",9.0,257.0,255.0,2.0,0.0,544.0,...,63.0,0.0,4.0,35.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2490,2490,2-Ingredient Coconut Yogurt,https://minimalistbaker.com/easy-2-ingredient-...,"[full_fat_coconut_milk, veganfriendly_probioti...",8.0,360.0,360.0,0.0,0.0,120.0,...,25.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2928,2928,Vegan Mint Chocolate Ice Cream Cake,https://minimalistbaker.com/vegan-mint-chocola...,"[bowl_vegan_chocolate_cake_recipe, vegan_mint_...",6.0,30.0,30.0,0.0,0.0,466.0,...,274.0,0.0,7.0,12.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
3072,3072,Vegan Brownie Chocolate Ice Cream,https://minimalistbaker.com/vegan-brownie-choc...,"[fullfat_coconut_milk, unsweetened_vanilla_alm...",3.0,150.0,150.0,0.0,0.0,208.0,...,0.0,0.0,9.0,15.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
1754,1754,Creamy Raspberry Chip Smoothie,https://minimalistbaker.com/creamy-raspberry-c...,"[dairyfree_milk, raspberry, banana, dairyfree_...",3.0,5.0,5.0,0.0,0.0,257.0,...,93.0,563.0,7.1,15.0,16.0,18.2,258.0,2.2,1.0,https://minimalistbaker.com/wp-content/uploads...
2904,2904,Vegan Roasted Red Pepper Pasta,https://minimalistbaker.com/vegan-roasted-red-...,"[bell_pepper, olive_oil, shallot, garlic, salt...",8.0,60.0,15.0,45.0,0.0,487.0,...,365.0,0.0,10.0,4.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
3042,3042,Vegan Snickers Ice Cream,https://minimalistbaker.com/vegan-snickers-ice...,"[fullfat_coconut_milk, sugar, vanilla_extract,...",7.0,45.0,30.0,15.0,0.0,361.0,...,36.0,0.0,2.8,23.4,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2250,2250,Seed Cycling Mixes,https://minimalistbaker.com/seed-cycling-mixes...,"[flax_seed, pumpkin_seed, chia_seed, desiccate...",4.0,10.0,10.0,0.0,0.0,98.0,...,4.0,99.0,2.3,0.4,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...


In [31]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136 # alfredmastann@gmail.com test user
user_item = table.get_item(Key={"user_id": int(user_id)})["Item"]

liked_idx = list(map(int, user_item["liked_idx"].keys()))
disliked_idx = list(map(int, user_item["disliked_idx"].keys()))

loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
predicted = loaded_model.predict([liked_idx, disliked_idx])

In [ ]:
liked_df = pd.DataFrame().from_dict(user_item["liked_idx"], orient="index").reset_index().rename(columns={"index": "recipe_id", 0: "date"})
liked_df["date"] = pd.to_datetime(liked_df["date"])
liked_df["recipe_id"] = liked_df["recipe_id"].astype(int)
liked_df.sort_values(by="date", ascending=False, inplace=True)
liked_df.reset_index(drop=True, inplace=True)
like_test = liked_df.iloc[:-5, :]
like_test

,recipe_id,date
0,1955,2025-08-11 17:47:31
1,4381,2025-08-11 17:47:25
2,4102,2025-07-22 22:25:17
3,5601,2025-07-22 22:25:12
4,995,2025-07-22 22:25:09
5,6081,2025-07-22 22:25:05
6,3220,2025-07-22 22:25:03
7,4005,2025-07-22 22:24:59
8,1076,2025-07-22 22:24:55
9,149,2025-07-22 22:24:50


In [35]:
disliked_df = pd.DataFrame().from_dict(user_item["disliked_idx"], orient="index").reset_index().rename(columns={"index": "recipe_id", 0: "date"})
disliked_df["date"] = pd.to_datetime(disliked_df["date"])
disliked_df["recipe_id"] = disliked_df["recipe_id"].astype(int)
disliked_df.sort_values(by="date", ascending=False, inplace=True)
disliked_df.reset_index(drop=True, inplace=True)
dislike_test = disliked_df.iloc[:-5, :]
dislike_test

,recipe_id,date
0,3934,2025-08-11 17:47:27
1,4006,2025-07-22 22:25:16
2,6228,2025-07-22 22:25:15
3,219,2025-07-22 22:25:14
4,6025,2025-07-22 22:25:13
5,1053,2025-07-22 22:25:08
6,5978,2025-07-22 22:25:07
7,930,2025-07-22 22:25:06
8,3193,2025-07-22 22:25:02
9,762,2025-07-22 22:25:01


In [12]:
prediction = loaded_model.predict([like_test["recipe_id"].values.tolist(), dislike_test["recipe_id"].values.tolist()])
prediction = np.asarray(json.loads(prediction))
prediction

array([4930, 3895, 3411, 5562, 5703, 3169, 3126, 3179, 5794, 5595, 3627,
       6131, 5585,  583,   90, 5301, 3400, 3149, 1496, 4241,  569, 3214,
       5892, 5499, 5685, 3830, 3211,  735, 5575,  756])

Recall@K (K = 10)

In [8]:
recall_k = np.isin(prediction, liked_df["recipe_id"].values.tolist()).sum()/len(liked_df)
recall_k

0.023809523809523808

Precision@K (K = 10)

In [9]:
precision_k = np.isin(prediction, liked_df["recipe_id"].values.tolist()).sum()/10
precision_k

0.1

### Registering

In [5]:
# Register model if validation is successful (RUN THIS TO UPDATE THE STREAMLIT MODEL)
with mlflow.start_run():
    mlflow.register_model(
        model_uri=model_info._model_uri,
        name="word2vec_model"
    )

Registered model 'word2vec_model' already exists. Creating a new version of this model...
2025/08/17 16:07:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: word2vec_model, version 12


🏃 View run charming-cub-981 at: http://127.0.0.1:5000/#/experiments/484303514770949339/runs/9a87b537f6314c6db73922d31945c24f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/484303514770949339


Created version '12' of model 'word2vec_model'.


### API Call Tests

In [14]:
params = {"query": ["rice", "beef", "meat"]}
response = requests.post(f"http://localhost:8000/recommend/", params=params)
response.text

'"[5297, 4557, 157, 161, 33, 4726, 5797, 274, 13, 618, 140, 14, 896, 251, 45, 40, 5068, 6047, 6073, 55, 206, 4709, 637, 231, 3863, 412, 5598, 3571, 5957, 307]"'